In [1]:
account_name = 'haj'
import datetime
login_str = 'login_'+account_name
import threading
import pandas as pd
from tqdm import tqdm
pd.set_option('display.max_rows', 105)
pd.set_option('display.max_columns', 100)
import numpy as np
import importlib
login_str = 'login_'+account_name
module = importlib.import_module(login_str, package=None)
pd.set_option('plotting.backend', 'pandas_bokeh')
import pandas_bokeh
pandas_bokeh.output_notebook()
print('Logging in with account : '+str(account_name))
module.retry_autologin()
import threading
from bokeh.io import export_png

Loading BokehJS ...

Logging in with account : haj
Trying to login...
access token : hQuOULw2YgrLYznWOQPUfu0PJ1FDDxA9
request token : 3Xr5AWfZtWpvcWe9nElou0zG2WVHd1oW


In [2]:
daysdiff = 45
brockerage_and_charges = 75
lotsize = 1000

In [3]:
class PnLCalculator:
    def __init__(self):
        self.quantity = 0
        self.cost = 0.0
        self.market_value = 0.0
        self.r_pnl = 0.0
        self.average_price = 0.0
        self.date = 0.0

    def fill(self, n_pos, exec_price,date):
        pos_change = n_pos - self.quantity
        direction = np.sign(pos_change)
        prev_direction = np.sign(self.quantity)
        qty_closing = min(abs(self.quantity), abs(pos_change)) * direction if prev_direction != direction else 0
        qty_opening = pos_change if prev_direction == direction else pos_change - qty_closing

        new_cost = self.cost + qty_opening * exec_price
        if self.quantity != 0:
            new_cost += qty_closing * self.cost / self.quantity
            self.r_pnl += qty_closing * (self.cost / self.quantity - exec_price)

        self.quantity = n_pos
        self.cost = new_cost
        self.date = date

    def update(self, price):
        if self.quantity != 0:
            self.average_price = self.cost / self.quantity
        else: 
            self.average_price = 0
        self.market_value = self.quantity * price
        return self.market_value - self.cost

In [4]:
ipdf_put = pd.read_excel('./PUT.xlsx')
insttkn_put = int(ipdf_put['token'][0])

ipdf_call = pd.read_excel('./CALL.xlsx')
insttkn_call = int(ipdf_call['token'][0])

interval='minute'
todaydt=datetime.date.today()
hud_ago=todaydt-datetime.timedelta(days=daysdiff)
to_date=datetime.date.isoformat(todaydt)
from_date=datetime.date.isoformat(hud_ago)

old_lst_put  = module.kite.historical_data(insttkn_put, from_date, to_date, interval,continuous=False)
mydf_put  = pd.DataFrame(old_lst_put)
mydf_put  = mydf_put.fillna(0)

old_lst_call  = module.kite.historical_data(insttkn_call, from_date, to_date, interval,continuous=False)
mydf_call  = pd.DataFrame(old_lst_call)
mydf_call  = mydf_call.fillna(0)


def backtester(filenumber,daysdiff,brockerage_and_charges,lotsize):
    '''backtester(filenumber,daysdiff,brockerage_and_charges,lotsize)'''
    ##############################################

    ipdf_put = pd.read_excel('./PUT'+str(filenumber)+'.xlsx')
    ipcsv_put = ipdf_put[['price','qty']]
    symbol_ip_put = ipdf_put['instrument'][0]
    insttkn_put = int(ipdf_put['token'][0])

    ipdf_call = pd.read_excel('./CALL'+str(filenumber)+'.xlsx')
    ipcsv_call = ipdf_call[['price','qty']]
    symbol_ip_call = ipdf_call['instrument'][0]
    insttkn_call = int(ipdf_call['token'][0])

    #######################################

    test_qty_list_put = []
    last_qty_put = 0
    for oneind in tqdm(range(mydf_put.index.size)):
        for qtyind in range(ipcsv_put.index.size):
            current_check_price = ipcsv_put.loc[qtyind]['price']
            if(current_check_price<=mydf_put.loc[oneind]['high'] and current_check_price>=mydf_put.loc[oneind]['low']):
                if(ipcsv_put.loc[qtyind]['qty'] != last_qty_put):
                    test_qty_list_put.append({'date':mydf_put.loc[oneind]['date'],
                                          'qty':ipcsv_put.loc[qtyind]['qty'],
                                          'price':ipcsv_put.loc[qtyind]['price'],
                                         })
                    last_qty_put = ipcsv_put.loc[qtyind]['qty']

    test_qty_list_call = []
    last_qty_call = 0
    for oneind in tqdm(range(mydf_call.index.size)):
        for qtyind in range(ipcsv_call.index.size):
            current_check_price = ipcsv_call.loc[qtyind]['price']
            if(current_check_price<=mydf_call.loc[oneind]['high'] and current_check_price>=mydf_call.loc[oneind]['low']):
                if(ipcsv_call.loc[qtyind]['qty'] != last_qty_call):
                    test_qty_list_call.append({'date':mydf_call.loc[oneind]['date'],
                                          'qty':ipcsv_call.loc[qtyind]['qty'],
                                          'price':ipcsv_call.loc[qtyind]['price'],
                                         })
                    last_qty_call = ipcsv_call.loc[qtyind]['qty']

    #########################################
    test_qty_list_df_put = pd.DataFrame(test_qty_list_put)
    test_qty_list_df_put['date'] = test_qty_list_df_put["date"].values.astype('datetime64')
    # print(test_qty_list_df_put)

    test_qty_list_df_call = pd.DataFrame(test_qty_list_call)
    test_qty_list_df_call['date'] = test_qty_list_df_call["date"].values.astype('datetime64')
    # print(test_qty_list_df_call)
    ###########################
    positions_put = test_qty_list_df_put['qty']
    exec_prices_put = test_qty_list_df_put['price']
    date_put = test_qty_list_df_put['date']
    pnls_put = []
    pos_put = PnLCalculator()
    pnls_put = []
    for (p,e,dt) in zip(positions_put, exec_prices_put,date_put):
        pos_put.fill(p, e,dt)
        u_pnl = pos_put.update(e)
        pnls_put.append({'date':pos_put.date,'Qty':int(pos_put.quantity),
                         'r_pnl':round(pos_put.r_pnl,4),
                         'u_pnl':round(u_pnl,4),
                         'total_pnl':round(pos_put.r_pnl+u_pnl,4),
                         'average_price':round(pos_put.average_price,4),
                        })

    pnls_df_put = pd.DataFrame(pnls_put)
    # pnls_df_put[['r_pnl','u_pnl','total_pnl']]=pnls_df_call[['r_pnl','u_pnl','total_pnl']]*lotsize
    # pnls_df_put[['date','r_pnl','u_pnl','total_pnl','Qty']].plot(x='date')



    ##########################

    positions_call = test_qty_list_df_call['qty']
    exec_prices_call = test_qty_list_df_call['price']
    date_call = test_qty_list_df_call['date']
    pnls_call = []
    pos_call = PnLCalculator()
    pnls_call = []
    for (p,e,dt) in zip(positions_call, exec_prices_call,date_call):
        pos_call.fill(p, e,dt)
        u_pnl = pos_call.update(e)
        pnls_call.append({'date':pos_call.date,'Qty':int(pos_call.quantity),
                          'r_pnl':round(pos_call.r_pnl,4),
                          'u_pnl':round(u_pnl,4),
                          'total_pnl':round(pos_call.r_pnl+u_pnl,4),
                          'average_price':round(pos_call.average_price,4),
                         })

    pnls_df_call = pd.DataFrame(pnls_call)
    # pnls_df_call[['r_pnl','u_pnl','total_pnl']]=pnls_df_call[['r_pnl','u_pnl','total_pnl']]*lotsize
    # pnls_df_call[['date','r_pnl','u_pnl','total_pnl','Qty']].plot(x='date')

    pnls_df_put = pnls_df_put.set_index('date')
    pnls_df_call = pnls_df_call.set_index('date')

    pnls_plot_df = pd.DataFrame()
    pnls_plot_df_put = pd.DataFrame()
    pnls_plot_df_call = pd.DataFrame()

    pnls_plot_df_put['put_pnl'] = pnls_df_put['total_pnl']
    pnls_plot_df_put['put_qty'] = pnls_df_put['Qty']
    pnls_plot_df_call['call_pnl'] = pnls_df_call['total_pnl']
    pnls_plot_df_call['call_qty'] = pnls_df_call['Qty']
    pnls_plot_df = pd.concat([pnls_plot_df_put,pnls_plot_df_call])
    # pnls_plot_df['Total_PNL'] = pnls_plot_df['put_pnl'] + pnls_plot_df['call_pnl']
    # pnls_plot_df = pnls_plot_df.fillna(0)

    gross_profit_put = pnls_put[-1]['total_pnl']*lotsize
    net_profit_put = pnls_put[-1]['total_pnl']*lotsize - ((brockerage_and_charges/2)*len(pnls_put))
    gross_profit_call = pnls_call[-1]['total_pnl']*lotsize
    net_profit_call = pnls_call[-1]['total_pnl']*lotsize - ((brockerage_and_charges/2)*len(pnls_call))
    print('*'*50)
    print('Put Final Values :')
    print(pnls_df_put[-1:])
    print('*'*50)
    print('Call Final Values :')
    print(pnls_df_call[-1:])
    # print('Total Charges : '+str((brockerage_and_charges/2)*len(pnls_call)))
    print('*'*50)
    print('Overall Performance :')
    print()
    print('Gross Profit Put : '+str(int(gross_profit_put)))
    print('Gross Profit Call : '+str(int(gross_profit_call)))
    print()
    print('Total Charges : '+str((brockerage_and_charges)*len(pnls_put)))
    print()
    print('net_profit_put : '+str(int(net_profit_put)))
    print('net_profit_call : '+str(int(net_profit_call)))
    print()
    print('Net Hedging Profit : '+str(int(net_profit_put+net_profit_call)))
    #############################################
    print('*'*65)
    print('Put Stastics :')
    print(pnls_df_put.describe())
    print('*'*65)
    print('Call Stastics :')
    print(pnls_df_call.describe())
    ######################################################


    SPY_Dat_put = pnls_df_put

    # window = 252
    # Roll_Max = SPY_Dat['total_pnl'].rolling(window, min_periods=1).max()
    # Daily_Drawdown = SPY_Dat['total_pnl']/Roll_Max - 1.0
    # Max_Daily_Drawdown = Daily_Drawdown.rolling(window, min_periods=1).min()

    Roll_Max_put = SPY_Dat_put['total_pnl'].cummax()
    Daily_Drawdown_put = SPY_Dat_put['total_pnl']/Roll_Max_put - 1.0
    Max_Daily_Drawdown_put = Daily_Drawdown_put.cummin()
    Drawdown_maindf_put = pd.DataFrame({'Daily_Drawdown_put':Daily_Drawdown_put,'Max_Daily_Drawdown_put':Max_Daily_Drawdown_put})


    ########################
    SPY_Dat_call = pnls_df_call

    Roll_Max_call = SPY_Dat_call['total_pnl'].cummax()
    Daily_Drawdown_call = SPY_Dat_call['total_pnl']/Roll_Max_call - 1.0
    Max_Daily_Drawdown_call = Daily_Drawdown_call.cummin()
    Drawdown_maindf_call = pd.DataFrame({'Daily_Drawdown_call':Daily_Drawdown_call,'Max_Daily_Drawdown_call':Max_Daily_Drawdown_call})

    dd_plot_df = pd.concat([Drawdown_maindf_put,Drawdown_maindf_call])
    ##################################################


    print('*'*50)
    R_put = pnls_df_put['total_pnl']
    r_put = R_put.diff()
    sr_put = r_put.mean()/r_put.std()
    sra_put = sr_put*np.sqrt(252/daysdiff)
    # print('Sharpe Ratio Put : '+str(round(sr_put,2)))
    print('Sharpe Ratio Annual Put : '+str(round(sra_put,2)))

    R_call = pnls_df_call['total_pnl']
    r_call = R_call.diff()
    sr_call = r_call.mean()/r_call.std()
    sra_call = sr_call*np.sqrt(252/daysdiff)
    # print('Sharpe Ratio Call : '+str(round(sr_call,2)))
    print('Sharpe Ratio Annual Call : '+str(round(sra_call,2)))
    print('Total Call + Put : '+str(round(sra_put+sra_call,2)))
    print('*'*50)

    plt1 = pnls_plot_df.plot()
    export_png(plt1, filename='plot_returns_'+str(filenumber)+'.png',height=700,width=1400)

    plt2 = dd_plot_df.plot()
    export_png(plt2, filename='plot_DD_'+str(filenumber)+'.png',height=700,width=1400)

    # Drawdown_maindf_put.plot()
    # Drawdown_maindf_call.plot()
    
    return{'put_final_val':pnls_df_put[-1:],'call_final_val':pnls_df_call[-1:],
           'gross_profit_put':gross_profit_put,'gross_profit_call':gross_profit_call,
           'net_profit_put':net_profit_put,'net_profit_call':net_profit_call,
           'total_charges':(brockerage_and_charges)*len(pnls_put),
           'net_hedging_profit':int(net_profit_put+net_profit_call),
           'put_stats':pnls_df_put.describe(),'call_stats':pnls_df_call.describe(),
           'sra_put':round(sra_put,2),'sra_call':round(sra_call,2),
           'Total_sra':round(sra_put+sra_call,2)
          }
    
# #     returns = 
    
#     print(pnls_df_put[-1:])
#     print(pnls_df_call[-1:])
#     print('Gross Profit Put : '+str(int(gross_profit_put)))
#     print('Gross Profit Call : '+str(int(gross_profit_call)))
    
#     print('Total Charges : '+str((brockerage_and_charges)*len(pnls_put)))
#     print('net_profit_put : '+str(int(net_profit_put)))
#     print('net_profit_call : '+str(int(net_profit_call)))
#     print('Net Hedging Profit : '+str(int(net_profit_put+net_profit_call)))
    
#     print(pnls_df_put.describe())
#     print(pnls_df_call.describe())
#     print('Sharpe Ratio Annual Put : '+str(round(sra_put,2)))
#     print('Sharpe Ratio Annual Call : '+str(round(sra_call,2)))
#     print('Total Call + Put : '+str(round(sra_put+sra_call,2)))
    
    

In [5]:
test_results = []

In [6]:
for i in range(10):
    ress = backtester(i+1,daysdiff,brockerage_and_charges,lotsize)
    test_results.append(ress)
    print(i+1)

100%|██████████| 14161/14161 [07:15<00:00, 32.48it/s]


**************************************************
Put Final Values :
                     Qty    r_pnl   u_pnl  total_pnl  average_price
date                                                               
2021-03-31 09:26:00  -56  44.1268  7.8732       52.0        73.7681
**************************************************
Call Final Values :
                     Qty  r_pnl  u_pnl  total_pnl  average_price
date                                                            
2021-03-31 08:40:00   16  34.25   1.75       36.0        73.3656
**************************************************
Overall Performance :

Gross Profit Put : 52000
Gross Profit Call : 36000

Total Charges : 2625

net_profit_put : 50687
net_profit_call : 34725

Net Hedging Profit : 85412
*****************************************************************
Put Stastics :
             Qty      r_pnl      u_pnl  total_pnl  average_price
count  35.000000  35.000000  35.000000  35.000000       35.00000
mean  -61.485714  18.51500

  0%|          | 0/14155 [00:00<?, ?it/s]

1


100%|██████████| 14161/14161 [07:21<00:00, 32.05it/s]


...

  0%|          | 0/14155 [00:00<?, ?it/s]

9


100%|██████████| 14161/14161 [07:10<00:00, 32.86it/s]


**************************************************
Put Final Values :
                     Qty    r_pnl    u_pnl  total_pnl  average_price
date                                                                
2021-03-31 09:26:00  -65  31.1532  12.0468       43.2        73.8003
**************************************************
Call Final Values :
                     Qty    r_pnl   u_pnl  total_pnl  average_price
date                                                               
2021-03-31 09:23:00   30  42.5495 -1.7495       40.8        73.4283
**************************************************
Overall Performance :

Gross Profit Put : 43200
Gross Profit Call : 40800

Total Charges : 4650

net_profit_put : 40875
net_profit_call : 37725

Net Hedging Profit : 78600
*****************************************************************
Put Stastics :
              Qty      r_pnl      u_pnl  total_pnl  average_price
count   62.000000  62.000000  62.000000  62.000000      62.000000
mean   -72.8

10


In [7]:
import joblib

In [8]:
joblib.dump(test_results,'test_results2.p')

['test_results.p']

In [66]:
test_res_analysis = []

In [67]:
for ij in range(len(test_results)):
    test_res_analysis.append({
        'put_final_Qty':test_results[ij]['put_final_val']['Qty'][0],
        'put_final_r_pnl':1000*test_results[ij]['put_final_val']['r_pnl'][0],
        'put_final_u_pnl':1000*test_results[ij]['put_final_val']['u_pnl'][0],
        'put_final_total_pnl':1000*test_results[ij]['put_final_val']['total_pnl'][0],
        'put_final_average_price':test_results[ij]['put_final_val']['average_price'][0],
        'call_final_Qty':test_results[ij]['call_final_val']['Qty'][0],
        'call_final_r_pnl':1000*test_results[ij]['call_final_val']['r_pnl'][0],
        'call_final_u_pnl':1000*test_results[ij]['call_final_val']['u_pnl'][0],
        'call_final_total_pnl':1000*test_results[ij]['call_final_val']['total_pnl'][0],
        'call_final_average_price':test_results[ij]['call_final_val']['average_price'][0],
        
        'gross_profit_put': test_results[ij]['gross_profit_put'],
        'gross_profit_call':  test_results[ij]['gross_profit_call'],
        'net_profit_put': test_results[ij]['net_profit_put'],
        'net_profit_call':  test_results[ij]['net_profit_call'],
        'total_charges':  test_results[ij]['total_charges'],
        'net_hedging_profit': test_results[ij]['net_hedging_profit'],
        
        'put_min_qty':test_results[ij]['put_stats']['Qty']['min'],
        'put_max_qty':test_results[ij]['put_stats']['Qty']['max'],
        'put_min_u_pnl':test_results[ij]['put_stats']['u_pnl']['min'],
        
        'call_min_qty':test_results[ij]['call_stats']['Qty']['min'],
        'call_max_qty':test_results[ij]['call_stats']['Qty']['max'],
        'call_min_u_pnl':test_results[ij]['call_stats']['u_pnl']['min'],
        
        'sra_put':test_results[ij]['sra_put'],
        'sra_call': test_results[ij]['sra_call'],
        'Total_sra': test_results[ij]['Total_sra'],
        
        
    })
    

                    
                    

In [58]:
test_res_analysis[0]

{'put_final_Qty': -56,
 'put_final_r_pnl': 44126.8,
 'put_final_u_pnl': 7873.2,
 'put_final_total_pnl': 52000.0,
 'put_final_average_price': 73.7681,
 'call_final_Qty': 16,
 'call_final_r_pnl': 34250.0,
 'call_final_u_pnl': 1750.0,
 'call_final_total_pnl': 36000.0,
 'call_final_average_price': 73.3656,
 'gross_profit_put': 52000.0,
 'gross_profit_call': 36000.0,
 'net_profit_put': 50687.5,
 'net_profit_call': 34725.0,
 'total_charges': 2625,
 'net_hedging_profit': 85412,
 'put_min_qty': -96.0,
 'put_max_qty': -40.0,
 'put_min_u_pnl': -76.0,
 'call_min_qty': -16.0,
 'call_max_qty': 32.0,
 'call_min_u_pnl': -12.0,
 'sra_put': 1.48,
 'sra_call': 3.63,
 'Total_sra': 5.11}

In [68]:
test_res_analysis_df = pd.DataFrame(test_res_analysis)
test_res_analysis_df

,put_final_Qty,put_final_r_pnl,put_final_u_pnl,put_final_total_pnl,put_final_average_price,call_final_Qty,call_final_r_pnl,call_final_u_pnl,call_final_total_pnl,call_final_average_price,gross_profit_put,gross_profit_call,net_profit_put,net_profit_call,total_charges,net_hedging_profit,put_min_qty,put_max_qty,put_min_u_pnl,call_min_qty,call_max_qty,call_min_u_pnl,sra_put,sra_call,Total_sra
0,-56,44126.8,7873.2,52000.0,73.7681,16,34250.0,1750.0,36000.0,73.3656,52000.0,36000.0,50687.5,34725.0,2625,85412,-96.0,-40.0,-76.0000,-16.0,32.0,-12.0000,1.48,3.63,5.11
1,-56,31119.9,10880.1,42000.0,73.7743,14,33810.0,1470.0,35280.0,73.3450,42000.0,35280.0,40837.5,33967.5,2325,74805,-91.0,-42.0,-70.5600,-21.0,28.0,-10.0800,1.40,3.49,4.89
2,-56,26169.8,2810.2,28980.0,73.8252,14,34011.2,1408.8,35420.0,73.3694,28980.0,35420.0,27817.5,34032.5,2325,61850,-91.0,-35.0,-80.5000,-21.0,28.0,-9.6600,1.03,3.48,4.50
3,-56,23882.2,9997.8,33880.0,73.8735,14,28644.0,2156.0,30800.0,73.3360,33880.0,30800.0,32605.0,29375.0,2550,61980,-91.0,-42.0,-71.9807,-21.0,35.0,-15.4000,1.06,3.25,4.31
4,-54,29336.4,9723.6,39060.0,73.8051,18,24535.0,3185.0,27720.0,73.3231,39060.0,27720.0,37635.0,26257.5,2850,63892,-90.0,-42.0,-74.3400,-12.0,36.0,-18.9000,1.21,2.55,3.76
5,0,4800.0,0.0,4800.0,0.0000,24,27240.7,359.3,27600.0,73.3150,4800.0,27600.0,4612.5,25912.5,375,30525,-12.0,0.0,-1.2000,-12.0,36.0,-16.8000,7.35,2.17,9.52
6,-60,27443.2,12456.8,39900.0,73.8426,24,40566.9,1613.1,42180.0,73.3028,39900.0,42180.0,38437.5,40005.0,2925,78442,-96.0,-48.0,-72.9600,-18.0,42.0,-18.4300,1.18,2.43,3.61
7,-60,35978.0,3622.0,39600.0,73.7554,25,32240.6,2859.4,35100.0,73.3156,39600.0,35100.0,37612.5,32775.0,3975,70387,-90.0,-45.0,-70.3837,-10.0,40.0,-18.5904,0.98,1.80,2.78
8,-65,25541.2,5058.8,30600.0,73.8228,30,29893.4,-143.4,29750.0,73.3148,30600.0,29750.0,28387.5,27200.0,4425,55587,-100.0,-50.0,-89.7576,-10.0,45.0,-19.1317,0.62,1.40,2.02
9,-65,31153.2,12046.8,43200.0,73.8003,30,42549.5,-1749.5,40800.0,73.4283,43200.0,40800.0,40875.0,37725.0,4650,78600,-105.0,-50.0,-89.9625,-10.0,45.0,-21.8014,0.82,1.55,2.38


In [61]:
ress1 = backtester(6,daysdiff,brockerage_and_charges,lotsize)

100%|██████████| 14161/14161 [07:28<00:00, 31.57it/s]


**************************************************
Put Final Values :
                     Qty    r_pnl   u_pnl  total_pnl  average_price
date                                                               
2021-03-31 08:45:00  -60  32.3394  2.4606       34.8         73.776
**************************************************
Call Final Values :
                     Qty    r_pnl   u_pnl  total_pnl  average_price
date                                                               
2021-03-31 09:26:00   24  27.2407  0.3593       27.6         73.315
**************************************************
Overall Performance :

Gross Profit Put : 34800
Gross Profit Call : 27600

Total Charges : 3975

net_profit_put : 32812
net_profit_call : 25912

Net Hedging Profit : 58725
*****************************************************************
Put Stastics :
             Qty      r_pnl      u_pnl  total_pnl  average_price
count  53.000000  53.000000  53.000000  53.000000      53.000000
mean  -62.377358 

In [62]:
test_results.append(ress1)

In [69]:
test_res_analysis_df.to_excel('./test_res_analysis_df.xlsx')